# JAM FITTER

In [1]:
import copy 
import numpy as np
import os
import pandas as pd
import pylab as py
import sys

from fitlab.mcsamp import MCSAMP
from fitlab.resman import RESMAN
from tools.config import load_config, conf
from tools.tools import load, save, checkdir

%matplotlib inline

In [2]:
def lprint(msg):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %msg)
    sys.stdout.flush()

### Runtime Options
Here I am specifying what type of run to do.

In [3]:
FIT_SIA = True 

## setup path to store the results 

In [4]:
path2results = 'sidis_transversity_with_sia_debug'
checkdir(path2results)

## data sets 

In [5]:
conf['datasets'] = {}

## SIDIS Collins Asymmetry

In [6]:
def load_sidis():
    conf['datasets']['sidis']={}
    conf['datasets']['sidis']['filters']={0:{'idx':None,'filter':None}}
    conf['datasets']['sidis']['filters'][0]['idx']=[4001,4000,4002,4004,4003,4005,3027,3025,3010]
    conf['datasets']['sidis']['filters'][0]['idx'].extend([3012,3005,3013,3026,3000,3003,3016,3004,3018])
    conf['datasets']['sidis']['filters'][0]['filter']="z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9"

    conf['datasets']['sidis']['xlsx']={}
    for dataset_index in conf['datasets']['sidis']['filters'][0]['idx']:
        conf["datasets"]["sidis"]["xlsx"][dataset_index] = 'sidis/expdata/%d.xlsx' % dataset_index     

    conf['datasets']['sidis']['norm']={}
    for k in conf['datasets']['sidis']['xlsx']: 
        conf['datasets']['sidis']['norm'][k]={'value':1,'fixed':True,'min':0,'max':1} 

In [7]:
def load_sia():
    conf['datasets']['sia']={}
    conf['datasets']['sia']['filters']={0:{'idx':None,'filter':None}}
    conf['datasets']['sia']['filters'][0]['idx']=[1000,1001,1002,1003,1004,1005,2008,2009]
    conf['datasets']['sia']['filters'][0]['filter']="Q2>1.69"
    
    conf['datasets']['sia']['xlsx']={}
    for dataset_index in conf['datasets']['sia']['filters'][0]['idx']:
        conf["datasets"]["sia"]["xlsx"][dataset_index] = 'sia/expdata/%d.xlsx' % dataset_index  

    conf['datasets']['sia']['norm']={}
    for k in conf['datasets']['sia']['xlsx']: 
        conf['datasets']['sia']['norm'][k]={'value':1,'fixed':True,'min':0,'max':1} 

In [8]:
load_sidis()

if FIT_SIA:
    load_sia()

## parameters

In [9]:
conf['params'] = {}
conf['shape'] = 1

### TMD PDF/FF parameters (from upol analysis)

In [10]:
conf['params'] = {}
conf['params']['pdf'] = {}
conf['params']['pdf']['widths0 valence']  = {'value':5.89294556274006398056e-01,'fixed':True,'min':0,'max':1}
conf['params']['pdf']['widths0 sea']      = {'value':6.33443286558464269120e-01,'fixed':True,'min':0,'max':1}

conf['params']['ff'] = {}
conf['params']['ff']['widths0 pi+ fav']   = {'value':1.15920500644793311729e-01,'fixed':True,'min':0,'max':1}
conf['params']['ff']['widths0 pi+ unfav'] = {'value':1.39782079427820671302e-01,'fixed':True,'min':0,'max':1}

### TMD Transversity parameters

In [11]:
conf['params']['transversity'] = {}
conf['params']['transversity']['widths0 valence'] = {'value':0.5,'fixed':False,'min':1e-5,'max':2}
conf['params']['transversity']['widths0 sea']     = {'value':0.0,'fixed':True,'min':1e-5,'max':2}
conf['params']['transversity']['u N']  = {'value':1,'fixed':False,'min':-10,'max':10}
conf['params']['transversity']['u a']  = {'value':-0.5,'fixed':False,'min':-1 ,'max':10}
conf['params']['transversity']['u b']  = {'value': 3.0,'fixed':False,'min': 1,'max':10}
conf['params']['transversity']['d N']  = {'value': 1.0,'fixed':False,'min':-20,'max':20}
conf['params']['transversity']['d a']  = {'value':-0.5,'fixed':False,'min':-1,'max':5}
conf['params']['transversity']['d b']  = {'value': 3.0,'fixed':False,'min':1e-5,'max':20}
conf['params']['transversity']['s N']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['s a']  = {'value':-0.5,'fixed':True,'min':-1,'max':5}
conf['params']['transversity']['s b']  = {'value': 3.0,'fixed':True,'min':1e-5,'max':10}
conf['params']['transversity']['u c']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['d c']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['s c']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['u d']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['d d']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
conf['params']['transversity']['s d']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

### TMD Collins parameters

In [12]:
def configure_collins_fixed():
    conf['params']['collins'] = {}
    conf['params']['collins']['widths0 pi+ fav']     = {'value':0.07249,'fixed':True,'min':1e-5,'max':1}
    conf['params']['collins']['widths0 pi+ unfav']   = {'value':0.10606,'fixed':True,'min':1e-5,'max':2}
    conf['params']['collins']['pi+ u N 1']  = {'value': 1.05988,'fixed':True,'min':0,'max':20}
    conf['params']['collins']['pi+ u a 1']  = {'value':-1.56637,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ u b 1']  = {'value': 4.76009 ,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ d N 1']  = {'value':-6.64443,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 1']  = {'value': 3.45852,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 1']  = {'value': 3.11503 ,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

    conf['params']['collins']['pi+ u N 2']  = {'value': 8.64876,'fixed':True,'min':0,'max':20}
    conf['params']['collins']['pi+ u a 2']  = {'value': 4.31424,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ u b 2']  = {'value': 3.59918,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ d N 2']  = {'value': 0.0,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 2']  = {'value': 0.0,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 2']  = {'value': 0.0,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

In [13]:
def configure_collins_free():
    conf['params']['collins']={}
    conf['params']['collins']['widths0 pi+ fav']     = {'value':7.47958632885192820083e-02,'fixed':False,'min':0.05,'max':0.10}
    conf['params']['collins']['widths0 pi+ unfav']   = {'value':0.059763576917398841815e+00,'fixed':False,'min':0.05,'max':0.136784756168045196212}
    conf['params']['collins']['pi+ u N 1']  = {'value': 4.49239476315589936206e+00,'fixed':False,'min':0,'max':3}
    conf['params']['collins']['pi+ u a 1']  = {'value':-8.28098465048009213518e-01,'fixed':False,'min':-1.9,'max':0}
    conf['params']['collins']['pi+ u b 1']  = {'value': 6.60780815284983358282e+00,'fixed':False,'min':2,'max':6}
    conf['params']['collins']['pi+ d N 1']  = {'value':-4.14852904854067539020e+00,'fixed':False,'min':-12,'max':0.0}
    conf['params']['collins']['pi+ d a 1']  = {'value': 1.00000000000000000000e+00,'fixed':False,'min': 0,'max':7.0}
    conf['params']['collins']['pi+ d b 1']  = {'value': 2.37348461151638101541e+00,'fixed':False,'min':2.5,'max':4.0}

    conf['params']['collins']['pi+ u c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

    conf['params']['collins']['pi+ u N 2']  = {'value': 6.11150078306284516572e+00,'fixed':False,'min':0,'max':15}
    conf['params']['collins']['pi+ u a 2']  = {'value': 4.71508421683099232524e+00,'fixed':False,'min':1,'max':8.0}
    conf['params']['collins']['pi+ u b 2']  = {'value': 2.75387728005980081392e+00,'fixed':False,'min':0,'max':5}

    conf['params']['collins']['pi+ d N 2']  = {'value': 0.0,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 2']  = {'value': 0.0,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 2']  = {'value': 0.0,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

In [14]:
if FIT_SIA:
    configure_collins_free()
else:
    configure_collins_fixed()

## Nested Sampling

### initialize fitpack libraries

In [15]:
conf['ncpus'] = 1
conf['resman'] = RESMAN()
mcsamp = MCSAMP()

loading sidis data sets 3013
multiprocess setup: ncpus=1 / observable
loading sia data sets 2009
multiprocess setup: ncpus=1 / observable


### start multiple NS runs
- the code will start n independent runs specified by size

In [16]:
conf['size']   = 8
conf['factor'] = 4 
mcsamp.run(path2results, 
           factor=conf['factor'], 
           size=conf['size'])

iter=1176  logz=-30.077 rel-err=2.792e-02  t-elapsed=4.036e+04  dchi2min=-2.262e+01 dchi2max=7.430e+01

Traceback (most recent call last):
Process Process-3:
Process Process-2:
Process Process-8:
Process Process-1:
Process Process-4:
Process Process-6:
Process Process-7:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", li

  File "/home/dmriser/repos/fitpack/fitlab/resman.py", line 100, in _get_residuals
    res,rres,nres=conf['resman'].get_residuals(par)
    _res,_rres,_nres=func()
  File "/home/dmriser/repos/fitpack/fitlab/resman.py", line 110, in get_residuals
    if 'sia'     in conf['datasets']: res,rres,nres=self._get_residuals(self.siares.get_residuals,res,rres,nres)
  File "/home/dmriser/repos/fitpack/fitlab/resman.py", line 110, in get_residuals
    if 'sia'     in conf['datasets']: res,rres,nres=self._get_residuals(self.siares.get_residuals,res,rres,nres)
  File "/home/dmriser/repos/fitpack/fitlab/resman.py", line 100, in _get_residuals
  File "/home/dmriser/repos/fitpack/tools/residuals.py", line 178, in get_residuals
    if 'sia'     in conf['datasets']: res,rres,nres=self._get_residuals(self.siares.get_residuals,res,rres,nres)
    _res,_rres,_nres=func()
  File "/home/dmriser/repos/fitpack/fitlab/resman.py", line 100, in _get_residuals
    self.get_theory()
  File "/home/dmriser/repos/fitpac

  File "/home/dmriser/repos/fitpack/qcdlib/tmdlib.py", line 58, in get_shape
    D1=conf[k1].get_C(z1,Q2,hadron1)
  File "/home/dmriser/repos/fitpack/tools/multiproc.py", line 24, in singlecore
    return self.singlecore()
    return  p[0]*x**p[1]*(1-x)**p[2]*(1+p[3]*x+p[4]*x**2)
  File "/home/dmriser/repos/fitpack/qcdlib/tmdlib.py", line 270, in get_C
    return [self.func(entry) for entry in self.data]
    self.get_theory()
KeyboardInterrupt
    return  p[0]*x**p[1]*(1-x)**p[2]*(1+p[3]*x+p[4]*x**2)
  File "/home/dmriser/repos/fitpack/tools/multiproc.py", line 24, in singlecore
  File "/home/dmriser/repos/fitpack/obslib/sia/residuals.py", line 48, in _get_theory
    C=self.get_collinear(z,hadron)#*ff
  File "/home/dmriser/repos/fitpack/tools/residuals.py", line 164, in get_theory
KeyboardInterrupt
  File "/home/dmriser/repos/fitpack/qcdlib/tmdlib.py", line 87, in get_collinear
    ZLcol=self.stfuncs.ZX(2,z1,z2,Q2,pT,h1+'+',h2+'+') + self.stfuncs.ZX(2,z1,z2,Q2,pT,h1+'-',h2+'-')
    out

KeyboardInterrupt: 

### combine multiple runs into one big run

In [ ]:
MC = mcsamp.get_MC_samples(path2results + '/mcdata')

### distribution of parameters

In [ ]:
samples  = MC['samples']
samples2 = MC['samples2']
weights  = MC['weights']
weights2 = MC['weights2']
order    = MC['order']
runs     = MC['runs']

nrows, ncols = 10, 3
fig = py.figure(figsize=(ncols*3,nrows*2))

def get_idx(key,tag):
    for i in range(len(order)):
        if order[i][1]==key and order[i][2]==tag:
            return i

global cnt
cnt=0
def plot(tags):
    global cnt
    for i in range(1,len(tags)):
        cnt+=1
        if tags[i]==None: continue
        idx=get_idx(tags[0],tags[i])
        ax=py.subplot(nrows,ncols,cnt)
        vmin=np.amin(samples2.T[idx])
        vmax=np.amax(samples2.T[idx])
        R=None#(vmin,vmax)
        for k in runs:
            ax.hist(runs[k]['samples'].T[idx],weights=runs[k]['weights'],bins=50,normed=True,range=R)
        ax.hist(samples.T[idx],weights=weights,bins=50,color='k',normed=True,range=R)
        ax.hist(samples2.T[idx],weights=weights2,bins=50,color='r',histtype='step',normed=True,range=R)
        ax.set_title(tags[i])
        mean=np.einsum('i,i',samples2.T[idx],weights2)
        std=np.einsum('i,i',(samples2.T[idx]-mean)**2,weights2)**0.5
        print '%20s = %10.5f +/- %10.5f'%(tags[i],mean,std)
        #ax.set_xlim(vmin,vmax)
    py.tight_layout()

plot(['transversity','widths0 valence','widths0 sea',None])
plot(['transversity','u N','u a','u b'])
plot(['transversity','d N','d a','d b'])
#plot(['collins','widths0 pi+ fav','widths0 pi+ unfav',None])
#plot(['collins','pi+ u N 1','pi+ u a 1','pi+ u b 1'])
#plot(['collins','pi+ d N 1','pi+ d a 1','pi+ d b 1'])

## data vs theory

In [ ]:
data={'weights':MC['weights2']}
cnt=0
for s in MC['samples2']:
    cnt+=1    
    lprint('%d/%d'%(cnt,len(MC['samples2'])))  
    conf['resman'].get_residuals(s);
    for k in conf['resman'].sidisres.tabs:
        if k  not in data: data[k]=[]
        thy=conf['resman'].sidisres.tabs[k]['thy']    
        norm=conf['datasets']['sidis']['norm'][k]['value']
        shift=conf['resman'].sidisres.tabs[k]['shift']        
        data[k].append(shift+thy/norm)
save(data,'%s/%s'%(path2results,'sidis.dat'))   

### compute averages and collect results

In [ ]:
data=load('%s/%s'%(path2results,'sidis.dat'))   
for k in data: data[k]=np.array(data[k])
thy,dthy={},{}
for k in data:
    if k=='weights': continue
    thy[k]=np.einsum('i,ik->k',data['weights'],data[k])
    dthy[k]=np.einsum('i,ik->k',data['weights'],(data[k]-thy[k])**2)**0.5
for k in thy: 
    conf['resman'].sidisres.tabs[k]['thy']=copy.copy(thy[k])
    conf['resman'].sidisres.tabs[k]['dthy']=copy.copy(dthy[k])

In [ ]:
report=conf['resman'].gen_report(verb=0,level=1)
delimiters=[]
for i in range(len(report)): 
    if 'reaction:' in report[i]: delimiters.append(i) 

data={}
nlines=len(report)
for i in range(len(delimiters)):
    ini=delimiters[i]
    if i==len(delimiters)-1: fin=len(report)
    else: fin=delimiters[i+1]
    reaction=report[ini].replace('reaction:','').strip()
    data[reaction]={'raw data':report[ini:fin]}
    
for k in data:
    print k
    block=data[k]['raw data']
    isep=[i for i in range(len(block)) if '--------' in block[i]][0]
    data[k]['summary']=[block[i] for i in range(isep)]
    data[k]['tables']=[block[i] for i in range(isep+1,len(block))]

    tabs={}
    for l in data[k]['tables']:
        info=l.split(',')
        col=[s for s in info if 'col' in s][0].split('=')[1].strip()
        if col not in tabs: tabs[col]={}
        info=[[ss.strip() for ss in s.split('=')] for s in info if 'col' not in info  if s.strip()!='']
        
        for s in info:
            if s[0] not in tabs[col]: tabs[col][s[0]]=[]
        
        for s in info:
            try:
                value=float(s[1])
            except:
                value=s[1]
            tabs[col][s[0]].append(value)        

    data[k]['tabs']=tabs
save(data,'%s/%s'%(path2results,'data_and_thy.dat'))  

In [ ]:
def summary():
    for k in data:
        print ""
        for l in data[k]['summary']: print l
summary()

### plot data and theory

In [ ]:
data=load('%s/%s'%(path2results,'data_and_thy.dat'))

nrows,ncols=3,3
fig = py.figure(figsize=(ncols*3,nrows*3))


tab=pd.DataFrame(data['sidis']['tabs']['HERMES']).query('tar=="proton"')
cnt=0
for dep in ['x','z','pT']:
    cnt+=1
    if dep=='pT': _dep='pt'
    else: _dep=dep
    ax=py.subplot(nrows,ncols,cnt)
    _tab=tab.query('dep=="%s"'%_dep)
    pip=_tab.query('had=="pi+"')
    pim=_tab.query('had=="pi-"')
    ax.errorbar(pip[dep],-pip['exp'],yerr=pip['alpha'],fmt='r.')
    ax.errorbar(pim[dep],-pim['exp'],yerr=pim['alpha'],fmt='b.')
    ax.plot(pip[dep],-pip['thy'],color='r',alpha=0.5)
    ax.plot(pim[dep],-pim['thy'],color='b',alpha=0.5)
    ax.fill_between(pip[dep],-(pip['thy']-pip['dthy']),-(pip['thy']+pip['dthy']),color='r',alpha=0.5)
    ax.fill_between(pim[dep],-(pim['thy']-pim['dthy']),-(pim['thy']+pim['dthy']),color='b',alpha=0.5)
    ax.set_xlabel(r'$%s$'%dep,size=20); ax.set_ylabel(r'$A$',size=20)
    ax.set_title('HERMES p')

    
tab=pd.DataFrame(data['sidis']['tabs']['compass']).query('tar=="proton"')
for dep in ['x','z','pT']:
    cnt+=1
    if dep=='pT': _dep='pt'
    else: _dep=dep
    ax=py.subplot(nrows,ncols,cnt)
    _tab=tab.query('dep=="%s"'%_dep)
    pip=_tab.query('had=="pi+"')
    pim=_tab.query('had=="pi-"')
    ax.errorbar(pip[dep],-pip['exp'],yerr=pip['alpha'],fmt='r.')
    ax.errorbar(pim[dep],-pim['exp'],yerr=pim['alpha'],fmt='b.')
    ax.plot(pip[dep],-pip['thy'],color='r',alpha=0.5)
    ax.plot(pim[dep],-pim['thy'],color='b',alpha=0.5)
    ax.fill_between(pip[dep],-(pip['thy']-pip['dthy']),-(pip['thy']+pip['dthy']),color='r',alpha=0.5)
    ax.fill_between(pim[dep],-(pim['thy']-pim['dthy']),-(pim['thy']+pim['dthy']),color='b',alpha=0.5)
    ax.set_xlabel(r'$%s$'%dep,size=20); ax.set_ylabel(r'$A$',size=20)
    ax.set_title('COMPASS p')

tab=pd.DataFrame(data['sidis']['tabs']['compass']).query('tar=="deuteron"')
for dep in ['x','z','pT']:
    cnt+=1
    if dep=='pT': _dep='pT'
    else: _dep=dep
    ax=py.subplot(nrows,ncols,cnt)
    _tab=tab.query('dep=="%s"'%_dep)
    pip=_tab.query('had=="pi+"')
    pim=_tab.query('had=="pi-"')
    ax.errorbar(pip[dep],-pip['exp'],yerr=pip['alpha'],fmt='r.')
    ax.errorbar(pim[dep],-pim['exp'],yerr=pim['alpha'],fmt='b.')
    ax.plot(pip[dep],-pip['thy'],color='r',alpha=0.5)
    ax.plot(pim[dep],-pim['thy'],color='b',alpha=0.5)
    ax.fill_between(pip[dep],-(pip['thy']-pip['dthy']),-(pip['thy']+pip['dthy']),color='r',alpha=0.5)
    ax.fill_between(pim[dep],-(pim['thy']-pim['dthy']),-(pim['thy']+pim['dthy']),color='b',alpha=0.5)
    ax.set_xlabel(r'$%s$'%dep,size=20); ax.set_ylabel(r'$A$',size=20)
    ax.set_title('COMPASS d')


py.tight_layout()

## PDFs and FFs

In [ ]:
def calc(func):
    RAW=[]
    cnt=0
    for s in MC['samples2']:
        cnt+=1    
        lprint('%d/%d'%(cnt,len(MC['samples2'])))  
        conf['parman'].set_new_params(s);    
        RAW.append(func())
    RAW=np.array(RAW)
    f =np.einsum('k,kif->if',MC['weights2'],RAW)
    df=np.einsum('k,kif->if',MC['weights2'],(RAW-f)**2)**0.5
    f=np.einsum('if->fi',f)
    df=np.einsum('if->fi',df)
    return {'f':f,'df':df}

In [ ]:
X1=10**np.linspace(-3,-1)
X2=np.linspace(0.101,0.999)
X=np.append(X1,X2)
h=calc(lambda : [conf['transversity'].get_C(x,1) for x in X])

In [ ]:
Z1=10**np.linspace(-3,-1)
Z2=np.linspace(0.101,0.999)
Z=np.append(Z1,Z2)
Hpi=calc(lambda : [conf['collins'].get_C(z,1,'pi+') for z in Z])

In [ ]:
nrows,ncols=1,1
py.figure(figsize=(ncols*5,nrows*5))
ax=py.subplot(nrows,ncols,1)
ax.fill_between(X,(h['f'][1]-h['df'][1]),(h['f'][1]+h['df'][1]),color='r',alpha=0.5)
ax.plot(X,h['f'][1],'r')
ax.fill_between(X,(h['f'][3]-h['df'][3]),(h['f'][3]+h['df'][3]),color='b',alpha=0.5)
ax.plot(X,h['f'][3],'b')
ax.set_ylabel(r'$h$',size=20)
ax.set_xlabel(r'$x$',size=20)
ax.set_ylim(-3,1.5)
ax.set_xlim(0,1)

#ax=py.subplot(nrows,ncols,2)
#ax.fill_between(Z,Z*(Hpi['f'][1]-Hpi['df'][1]),Z*(Hpi['f'][1]+Hpi['df'][1]),color='r',alpha=0.5)
#ax.fill_between(Z,Z*(Hpi['f'][3]-Hpi['df'][3]),Z*(Hpi['f'][3]+Hpi['df'][3]),color='b',alpha=0.5)
#ax.plot(Z,Z*Hpi['f'][1],'r')
#ax.plot(Z,Z*Hpi['f'][3],'b')

#ax.set_ylabel(r'$H$',size=20)
#ax.set_xlabel(r'$x$',size=20)
#ax.set_ylim(-0.5,0.5)

py.tight_layout()